In [1]:
import numpy as np
# import neuralNetwork 
from os.path  import join
from datasetLoad import MnistDataloader

28


In [9]:

#input layer
class inputLayer:

    def __init__(self, number_of_input): 
        self.number_of_input = number_of_input
    
    #to add the vlaues in the network 
    def use(self, input_matrix):
        if isinstance(input_matrix,np.ndarray):
            self.out_matrix = input_matrix
            return input_matrix
        else:
            self.out_matrix = np.array(input_matrix)
            return self.out_matrix
    
    def fetchInputValues(self):
        return self.out_matrix

#hidden and output layers
class layers:
    
    def __init__(self, number_of_input, number_of_neurons, activation='relu'):
        low, high = -1, 1
        self.weight_matrix = np.random.uniform(low, high, size=(number_of_input, number_of_neurons))
        self.bias = np.random.uniform(low, high, size=(1, number_of_neurons))
        self.activation = activation

    def apply_activation(self, values):
        if self.activation == 'relu':
            return np.maximum(0, values)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-values))
        elif self.activation == 'tanh':
            return np.tanh(values)
        elif self.activation == 'softmax':
            e_x = np.exp(values - np.max(values, axis=-1, keepdims=True))
            return e_x / np.sum(e_x, axis=-1, keepdims=True)
        else:
            return values  # No activation (identity)

    def compute_derivative(self, values):
        if self.activation == 'relu':
            return (values > 0).astype(float)
        elif self.activation == 'sigmoid':
            sigmoid_output = self.apply_activation(values)
            return sigmoid_output * (1 - sigmoid_output)
        elif self.activation == 'tanh':
            tanh_output = self.apply_activation(values)
            return 1 - tanh_output**2
        elif self.activation == 'softmax':
            softmax_output = self.apply_activation(values)
            return softmax_output * (1 - softmax_output)  # Note: This derivative might not be used often for softmax
        else:
            return np.ones_like(values)  # Derivative of the identity function is 1

    def forward(self, input_matrix):
        print(f'shapeofIN = {np.shape(input_matrix)}, shapeofWI = {np.shape(self.weight_matrix)}') #debugging
        values = np.dot(input_matrix,self.weight_matrix) + self.bias
        self.neuronValues =  self.apply_activation(values)
        # print(self.neuronValues) #debugging
        return self.neuronValues
    
class Model:

    #to intiallize the model
    def __init__(self,):
        inputNumber = int(input('enter the input parameters: ')) # input nodes
        self.numberOfLayers = int(input('enter the number of layers: ')) #number of hidden layer and output layer

        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = int(input('number of input: '))
            numberOfNeurons = int(input('number of neurons: '))
            activation = input('Enter the activation funtion: ')
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = data
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        # print(layers.bias) #debugging
        i -= 1

        while i:
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = layers.compute_derivative(newValues)
            cur_gradient = np.multiply(derValues.T,pre_w)
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            layers.weight_matrix = upWeights
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            # print(layers.bias) #debugging

            i -= 1

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = layers.neuronValue
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = data
        weights = layer.weight_matrix
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        layers.bias = layers.bias - learningRate*(pr)
    
class ModelTest:

    #to intiallize the model
    def __init__(self,parameters):
        ind = 0
        inputNumber = parameters[ind] # input nodes
        ind += 1
        self.numberOfLayers = parameters[ind] #number of hidden layer and output layer
        ind += 1
        self.listOfHiddenLayers = []
        self.inLayerObj = inputLayer(inputNumber)

        for i in range(self.numberOfLayers):
            number_of_input = parameters[ind]
            ind += 1 
            numberOfNeurons = parameters[ind]
            ind += 1
            activation = parameters[ind]
            ind += 1
            hidden_layer_obj = layers(number_of_input,numberOfNeurons,activation)

            self.listOfHiddenLayers.append(hidden_layer_obj)
            
            if i == self.numberOfLayers-1:
                self.numberOfLabels = numberOfNeurons

    def pridict(self,data,ForTraining = False):

        arr = data
        
        for layer in self.listOfHiddenLayers:
            arr = layer.forward(arr)

        if ForTraining:
            return arr

        pridiction_value = -1
        pridiction = None

        for i,obj in enumerate(arr[0]):
            if pridiction_value < obj:
                pridiction = {'label' : i, 'confidence' : obj}
        
        return pridiction
    

    def view_weights(self):
        for layer in self.listOfHiddenLayers:
            print(f'weights \n - {layer.weight_matrix}')
            print(f'bias - {layer.bias}')

    #backpropagation 
    def train(self,data,label,learningRate=0.1):
        print('here') #debugging
        pridiction = self.pridict(data,ForTraining = True)
        reqArr =np.array([0 for i in range(self.numberOfLabels)])
        reqArr[label] = 1
        costFuntion = sum((pridiction - reqArr)**2)

        pridiction = pridiction - reqArr
        i = self.numberOfLayers - 1

        layers = self.listOfHiddenLayers[i]
        gradient = np.multiply(pridiction,layers.compute_derivative(pridiction))
        prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
        pr = layers.weight_matrix.T - learningRate*(np.dot(gradient.T,prevNuronValues))
        layers.weight_matrix = pr.T
        layers.bias = layers.bias - learningRate*(gradient)
        # print(layers.bias) #debugging
        i -= 1

        while i:
            
            #to get gradient 
            layers = self.listOfHiddenLayers[i]
            pre_w = self.listOfHiddenLayers[i+1].weight_matrix
            newValues = layers.neuronValues
            # print(f'newValues = {i} - {newValues}') #debugging
            derValues = layers.compute_derivative(newValues)
            cur_gradient = np.multiply(derValues.T,pre_w)
            pr = np.dot(gradient,cur_gradient.T)
            prevNuronValues = self.listOfHiddenLayers[i-1].neuronValues
            
            #updating weights
            weights = layers.weight_matrix
            upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
            print(f'wei = {np.shape(upWeights)}') #debugging
            layers.weight_matrix = upWeights
            gradient = pr
            
            #updating the bias 
            layers.bias = layers.bias - learningRate*(pr)
            # print(layers.bias) #debugging

            i -= 1

        #for input layer - first layer value
        layer = self.listOfHiddenLayers[0]
        pre_w = self.listOfHiddenLayers[1].weight_matrix
        newValues = self.listOfHiddenLayers[1].neuronValues
        print(f'new = {np.shape(newValues)}') #debugging
        derValues = layers.compute_derivative(newValues)
        cur_gradient = np.dot(pre_w,gradient.T)
        print(f'cur_gra = {np.shape(cur_gradient)}') #debugging
        pr = np.multiply(derValues,cur_gradient.T)
        prevNuronValues = data
        weights = layer.weight_matrix
        print(f'weight = {np.shape(weights)}') #debugging
        upWeights = weights.T - learningRate*(np.dot(pr.T,prevNuronValues))
        layers.weight_matrix = upWeights.T
        layers.bias = layers.bias - learningRate*(pr)

In [2]:
input_path = r'C:\programs\projects\NN\input'
training_images_filepath = join(input_path, 'train-images-idx3-ubyte/train-images-idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels-idx1-ubyte/train-labels-idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images-idx3-ubyte/t10k-images-idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte')


mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

x_train = np.array(x_train)
x_test = np.array(x_test)

# print(x_test[1],y_test[1])

In [10]:
# Flatten the training and test images
x_train_flat = x_train.reshape(x_train.shape[0], -1)  
x_test_flat = x_test.reshape(x_test.shape[0], -1)     

testInputs = [784 ,3,784,16,'relu',16,16,'relu',16,10,'sigmoid']
model = ModelTest(testInputs)

'''
input parameters = 784
number of layers = 3

layer 1:
number of input: 784
number of neurons: 16
Enter the activation funtion: relu

layer 2:
number of input: 16
number of neurons: 16
Enter the activation funtion: relu

layer 3:
number of input: 16
number of neurons: 10
Enter the activation funtion: sigmoid
'''

'\ninput parameters = 784\nnumber of layers = 3\n\nlayer 1:\nnumber of input: 784\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 2:\nnumber of input: 16\nnumber of neurons: 16\nEnter the activation funtion: relu\n\nlayer 3:\nnumber of input: 16\nnumber of neurons: 10\nEnter the activation funtion: sigmoid\n'

In [11]:
for x, y in zip(x_train_flat, y_train):
    x = np.array([x])
    model.train(x, y)


here
shapeofIN = (1, 784), shapeofWI = (784, 16)
shapeofIN = (1, 16), shapeofWI = (16, 16)
shapeofIN = (1, 16), shapeofWI = (16, 10)
wei = (16, 16)
new = (1, 16)
cur_gra = [[-4.55607717e+05]
 [-2.39228741e+05]
 [-9.50213407e+05]
 [-1.67630505e+01]
 [-1.00377247e+02]
 [-8.74004025e+05]
 [-3.31548240e+05]
 [-7.46466287e+05]
 [-1.70514483e+04]
 [ 1.06842581e+01]
 [-1.09262789e+02]
 [-5.31437349e+05]
 [ 6.50431649e+00]
 [-4.22379704e+01]
 [-3.94271890e+01]
 [-8.49508759e+05]]
weight = [[ 0.2016439  -0.12789001  0.12207099 ...  0.70237823 -0.73677803
  -0.93113891]
 [-0.24463978 -0.26459953  0.45020729 ... -0.02833805  0.57157274
   0.63737724]
 [-0.0929164  -0.53742713  0.88112299 ...  0.44064022  0.25186576
  -0.08356085]
 ...
 [ 0.33649506  0.30422283 -0.31868325 ... -0.77895707 -0.57609631
  -0.60621024]
 [-0.68505383  0.65114995  0.00712311 ...  0.89653097 -0.16972521
  -0.34070558]
 [-0.246392   -0.91191439  0.69724719 ...  0.78749062 -0.17976716
   0.81363246]]
here
shapeofIN = (1, 7

ValueError: shapes (1,16) and (784,16) not aligned: 16 (dim 1) != 784 (dim 0)

In [ ]:
a = np.array([x_train_flat[0]])
model.pridict(a)

[[ 312.70795421    0.          659.66035991    0.            0.
   202.32734049    0.            0.            0.            0.
   796.61605486 1666.14526113 1366.73697261  405.05207695   63.8154843
     0.        ]]
[[1244.67081574    0.           50.4632191     0.            0.
     0.          389.93838025    0.            0.          791.08213188
  1399.00574794    0.            0.          261.57572019  914.36981593
     0.        ]]
[[0.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 1.70889831e-309 0.00000000e+000
  0.00000000e+000 0.00000000e+000]]


{'label': 9, 'confidence': 0.0}